In [1]:
from train import train

args = {
    'batch_size': 128,
    'model': 'seresnext50',
    'freeze': False,
    'eval_all': True,
    'data_root': '../dataset/classify-leaves',
    'val_ratio': 0.1,
    'fold': 10,
    'lr': 2e-5,
    'epoch': 20,
    'resume': True,
    'verbose': True,
    'ckpt_path': 'model.pth'
}
class AttributeDict(dict):
    __slots__ = () 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
args = AttributeDict(args)

train(args)

Finished reading the train set of Leaves Dataset (25856 samples found)
Training for fold 0/10...
loaded model from /home/liyiming/projects/classify-leaves/src/../models/fold=0-model.pth for resume training with current_epoch=5
Epoch 5/30 training...


  0%|          | 0/182 [00:00<?, ?it/s]

9/182 loss:2.4234832525253296  acc:0.5109375
19/182 loss:2.421932888031006  acc:0.50859375
29/182 loss:2.431186040242513  acc:0.5015625
39/182 loss:2.4389314889907836  acc:0.5
49/182 loss:2.443057446479797  acc:0.500625
59/182 loss:2.4513481775919597  acc:0.4973958333333333
69/182 loss:2.4515716927392144  acc:0.4987723214285714
79/182 loss:2.4487174332141874  acc:0.5005859375
89/182 loss:2.4435309383604262  acc:0.5027777777777778
99/182 loss:2.440350823402405  acc:0.50421875
109/182 loss:2.436165486682545  acc:0.5055397727272727
119/182 loss:2.4344004452228547  acc:0.5064453125
129/182 loss:2.4295913494550265  acc:0.5086538461538461
139/182 loss:2.427437060219901  acc:0.5090401785714286
149/182 loss:2.4230031696955363  acc:0.5106770833333333
159/182 loss:2.417158584296703  acc:0.512890625
169/182 loss:2.413730103829328  acc:0.5140165441176471
179/182 loss:2.412342949708303  acc:0.5146701388888889
train loss:2.4123753054643013 acc:0.5146110872367856
acc:0.20030935811079112 acc5:0.450116

  0%|          | 0/182 [00:00<?, ?it/s]

9/182 loss:2.424180030822754  acc:0.51328125
19/182 loss:2.3959267258644106  acc:0.5234375
29/182 loss:2.403916851679484  acc:0.5203125
39/182 loss:2.407920604944229  acc:0.5255859375
49/182 loss:2.3904698991775515  acc:0.5321875
59/182 loss:2.39357369740804  acc:0.5282552083333333
69/182 loss:2.3984215668269564  acc:0.5262276785714286
79/182 loss:2.403048488497734  acc:0.523828125
89/182 loss:2.4031734704971313  acc:0.5237847222222223
99/182 loss:2.4033632612228395  acc:0.52421875
109/182 loss:2.407884138280695  acc:0.5220170454545454
119/182 loss:2.4109029630819956  acc:0.5220052083333333
129/182 loss:2.412811515881465  acc:0.5192307692307693
139/182 loss:2.4097404939787728  acc:0.5196986607142857
149/182 loss:2.406605839729309  acc:0.52125
159/182 loss:2.4079025983810425  acc:0.5224609375
169/182 loss:2.4037432782790242  acc:0.5232077205882353
179/182 loss:2.4039870884683396  acc:0.5227864583333334
train loss:2.403956868917992 acc:0.5226042114310271
acc:0.12335653522405529 acc5:0.29

...

9/182 loss:2.263192129135132  acc:0.58125
19/182 loss:2.2510167479515077  acc:0.578515625
29/182 loss:2.28202699025472  acc:0.5674479166666667
39/182 loss:2.2769985437393188  acc:0.5666015625
49/182 loss:2.271022663116455  acc:0.56890625
59/182 loss:2.279079329967499  acc:0.567578125
69/182 loss:2.280478290149144  acc:0.5646205357142857
79/182 loss:2.284757587313652  acc:0.563671875
89/182 loss:2.2720733245213824  acc:0.5675347222222222
99/182 loss:2.2727071237564087  acc:0.565859375
109/182 loss:2.267759318785234  acc:0.5674715909090909
119/182 loss:2.2663990199565887  acc:0.5688802083333333
129/182 loss:2.26681028512808  acc:0.5685697115384616
139/182 loss:2.260250571795872  acc:0.5706473214285714
149/182 loss:2.256757197380066  acc:0.5723958333333333
159/182 loss:2.254343159496784  acc:0.572021484375
169/182 loss:2.2527081181021296  acc:0.5720588235294117
179/182 loss:2.2544675641589693  acc:0.5709635416666666
train loss:2.2549778222320516 acc:0.5708207993124195
acc:0.24091260642251

  0%|          | 0/182 [00:00<?, ?it/s]

9/182 loss:2.2707129240036013  acc:0.5671875


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>